In [1]:
import os, sys
sys.path.append(os.getcwd())

import tflib.save_images
import tflib.wikiartGenre
import tflib.plot
import imageio
from functools import reduce

from GANgogh import *

from tflib.wikiartGenre import get_style

DEVICES = ['/gpu:{}'.format(i) for i in range(N_GPUS)]


Uppercase local vars:
	BATCH_SIZE: 84
	CLASSES: 2
	CRITIC_ITERS: 5
	DIM: 64
	LAMBDA: 10
	MODE: acwgan
	N_GPUS: 1
	OUTPUT_DIM: 12288


In [2]:
import tensorflow as tf

In [3]:
tf.__version__

'1.15.5'

In [4]:
# pip install jupyterlab-nvdashboard

In [5]:
import tensorflow as tf
print("# GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# GPUs Available:  1


In [ ]:
# pip install nvidia-smi

In [ ]:
# import nvidia-smi

In [ ]:
tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [ ]:
conda list cudatoolkit

In [ ]:
conda list cudnn

In [22]:
ITERS = 2000 #200000  # How many iterations to train for
PREITERATIONS = 20# 2000  # Number of preiteration training cycles to run


Generator, Discriminator = GeneratorAndDiscriminator()

with tf.Session(config=tf.ConfigProto(allow_soft_placement=True)) as session:
    all_real_data_conv = tf.placeholder(tf.int32, shape=[BATCH_SIZE, 3, 64, 64])
#     print(all_real_data_conv)
    all_real_label_conv = tf.placeholder(tf.int32, shape=[BATCH_SIZE, CLASSES])

    generated_labels_conv = tf.placeholder(tf.int32, shape=[BATCH_SIZE, CLASSES])
    sample_labels_conv = tf.placeholder(tf.int32, shape=[BATCH_SIZE, CLASSES])

    split_real_data_conv = tf.split(all_real_data_conv, len(DEVICES))
    split_real_label_conv = tf.split(all_real_label_conv, len(DEVICES))
    split_generated_labels_conv = tf.split(generated_labels_conv, len(DEVICES))
    split_sample_labels_conv = tf.split(sample_labels_conv, len(DEVICES))

    gen_costs, disc_costs = [], []

    for device_index, (device, real_data_conv, real_label_conv) in enumerate(
            zip(DEVICES, split_real_data_conv, split_real_label_conv)):

        with tf.device(device):
            real_data = tf.reshape(2 * ((tf.cast(real_data_conv, tf.float32) / 255.) - .5),
                                   [BATCH_SIZE // len(DEVICES), OUTPUT_DIM])
            real_labels = tf.reshape(real_label_conv, [BATCH_SIZE // len(DEVICES), CLASSES])

            generated_labels = tf.reshape(split_generated_labels_conv, [BATCH_SIZE // len(DEVICES), CLASSES])
            sample_labels = tf.reshape(split_sample_labels_conv, [BATCH_SIZE // len(DEVICES), CLASSES])

            fake_data, fake_labels = Generator(BATCH_SIZE // len(DEVICES), CLASSES, generated_labels)

            # set up discrimnator results

            disc_fake, disc_fake_class = Discriminator(fake_data, CLASSES)
            disc_real, disc_real_class = Discriminator(real_data, CLASSES)

            prediction = tf.argmax(disc_fake_class, 1)
            correct_answer = tf.argmax(fake_labels, 1)
            equality = tf.equal(prediction, correct_answer)
            genAccuracy = tf.reduce_mean(tf.cast(equality, tf.float32))

            prediction = tf.argmax(disc_real_class, 1)
            correct_answer = tf.argmax(real_labels, 1)
            equality = tf.equal(prediction, correct_answer)
            realAccuracy = tf.reduce_mean(tf.cast(equality, tf.float32))

            gen_cost = -tf.reduce_mean(disc_fake)
            disc_cost = tf.reduce_mean(disc_fake) - tf.reduce_mean(disc_real)

            gen_cost_test = -tf.reduce_mean(disc_fake)
            disc_cost_test = tf.reduce_mean(disc_fake) - tf.reduce_mean(disc_real)

            generated_class_cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=disc_fake_class,
                                                                                          labels=fake_labels))

            real_class_cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=disc_real_class,
                                                                                     labels=real_labels))
            gen_cost += generated_class_cost
            disc_cost += real_class_cost

            alpha = tf.random_uniform(
                shape=[BATCH_SIZE // len(DEVICES), 1],
                minval=0.,
                maxval=1.
            )
            differences = fake_data - real_data
            interpolates = real_data + (alpha * differences)
            gradients = tf.gradients(Discriminator(interpolates, CLASSES)[0], [interpolates])[0]
            slopes = tf.sqrt(tf.reduce_sum(tf.square(gradients), reduction_indices=[1]))
            gradient_penalty = tf.reduce_mean((slopes - 1.) ** 2)
            disc_cost += LAMBDA * gradient_penalty

            real_class_cost_gradient = real_class_cost * 50 + LAMBDA * gradient_penalty

            gen_costs.append(gen_cost)
            disc_costs.append(disc_cost)

    gen_cost = tf.add_n(gen_costs) / len(DEVICES)
    disc_cost = tf.add_n(disc_costs) / len(DEVICES)

    gen_train_op = tf.train.AdamOptimizer(learning_rate=1e-4, beta1=0.5, beta2=0.9).minimize(gen_cost,
                                                                                             var_list=lib.params_with_name(
                                                                                                 'Generator'),
                                                                                             colocate_gradients_with_ops=True)
    disc_train_op = tf.train.AdamOptimizer(learning_rate=1e-4, beta1=0.5, beta2=0.9).minimize(disc_cost,
                                                                                              var_list=lib.params_with_name(
                                                                                                  'Discriminator.'),
                                                                                              colocate_gradients_with_ops=True)
    class_train_op = tf.train.AdamOptimizer(learning_rate=1e-4, beta1=0.5, beta2=0.9).minimize(real_class_cost_gradient,
                                                                                               var_list=lib.params_with_name(
                                                                                                   'Discriminator.'),
                                                                                               colocate_gradients_with_ops=True)
    # For generating samples

    fixed_noise = tf.constant(np.random.normal(size=(BATCH_SIZE, 128)).astype('float32'))
    all_fixed_noise_samples = []
    for device_index, device in enumerate(DEVICES):
        n_samples = BATCH_SIZE // len(DEVICES)
        all_fixed_noise_samples.append(Generator(n_samples, CLASSES, sample_labels, noise=fixed_noise[
                                                                                          device_index * n_samples:(
                                                                                                                               device_index + 1) * n_samples])[
                                           0])
        if tf.__version__.startswith('1.'):
            all_fixed_noise_samples = tf.concat(all_fixed_noise_samples, axis=0)
        else:
            all_fixed_noise_samples = tf.concat(0, all_fixed_noise_samples)


    def generate_image(iteration):
        for i in range(CLASSES):
            curLabel = genRandomLabels(BATCH_SIZE, CLASSES, condition=i)
            samples = session.run(all_fixed_noise_samples, feed_dict={sample_labels: curLabel})
            samples = ((samples + 1.) * (255.99 / 2)).astype('int32')
            save_images(samples.reshape((BATCH_SIZE, 3, 64, 64)),
                                        'generated/samples_{}_{}.png'.format(get_style(i), iteration))


    # Dataset iterator
    train_gen, dev_gen = lib.wikiartGenre.load(BATCH_SIZE, CLASSES)


    def softmax_cross_entropy(logit, y):
        return -tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logit, labels=y))


    def inf_train_gen():
        while True:
            for (images, labels) in train_gen():
                yield images, labels


    _sample_labels = genRandomLabels(BATCH_SIZE, CLASSES)
    # Save a batch of ground-truth samples
    _x, _y = next(train_gen())
#     print(_x, _y)
    _x_r = session.run(real_data, feed_dict={all_real_data_conv: _x})
    print({all_real_data_conv: _x})
    
    _x_r = ((_x_r + 1.) * (255.99 / 2)).astype('int32')
#     imageio.imwrite('generated/samples_groundtruth.png',_x_r)
    save_images(_x_r.reshape((BATCH_SIZE, 3, 64, 64)), 'generated/samples_groundtruth.png')
#     imageio.imwrite('generated/samples_groundtruth.png',_x_r)

    session.run(tf.initialize_all_variables(), feed_dict={generated_labels_conv: genRandomLabels(BATCH_SIZE, CLASSES)})
    gen = train_gen()

    for iterp in range(PREITERATIONS):
        _data, _labels = next(gen)
        _, accuracy = session.run([disc_train_op, realAccuracy],
                                  feed_dict={all_real_data_conv: _data, all_real_label_conv: _labels,
                                             generated_labels_conv: genRandomLabels(BATCH_SIZE, CLASSES)})
        if iterp % 100 == 99:
            print('pretraining accuracy: ' + str(accuracy))

    for iteration in range(ITERS):
        start_time = time.time()
        # Train generator
        if iteration > 0:
            _ = session.run(gen_train_op, feed_dict={generated_labels_conv: genRandomLabels(BATCH_SIZE, CLASSES)})
        # Train critic
        disc_iters = CRITIC_ITERS
        for i in range(disc_iters):
            _data, _labels = next(gen)
            _disc_cost, _disc_cost_test, class_cost_test, gen_class_cost, _gen_cost_test, _genAccuracy, _realAccuracy, _ = session.run(
                [disc_cost, disc_cost_test, real_class_cost, generated_class_cost, gen_cost_test, genAccuracy,
                 realAccuracy, disc_train_op], feed_dict={all_real_data_conv: _data, all_real_label_conv: _labels,
                                                          generated_labels_conv: genRandomLabels(BATCH_SIZE, CLASSES)})

        lib.plot.plot('train disc cost', _disc_cost)
        lib.plot.plot('time', time.time() - start_time)
        lib.plot.plot('wgan train disc cost', _disc_cost_test)
        lib.plot.plot('train class cost', class_cost_test)
        lib.plot.plot('generated class cost', gen_class_cost)
        lib.plot.plot('gen cost cost', _gen_cost_test)
        lib.plot.plot('gen accuracy', _genAccuracy)
        lib.plot.plot('real accuracy', _realAccuracy)

        if (iteration % 100 == 99 and iteration < 1000) or iteration % 1000 == 999:
            t = time.time()
            dev_disc_costs = []
            images, labels = next(dev_gen())
            _dev_disc_cost, _dev_disc_cost_test, _class_cost_test, _gen_class_cost, _dev_gen_cost_test, _dev_genAccuracy, _dev_realAccuracy = session.run(
                [disc_cost, disc_cost_test, real_class_cost, generated_class_cost, gen_cost_test, genAccuracy,
                 realAccuracy], feed_dict={all_real_data_conv: images, all_real_label_conv: labels,
                                           generated_labels_conv: genRandomLabels(BATCH_SIZE, CLASSES)})
            dev_disc_costs.append(_dev_disc_cost)
            lib.plot.plot('dev disc cost', np.mean(dev_disc_costs))
            lib.plot.plot('wgan dev disc cost', _dev_disc_cost_test)
            lib.plot.plot('dev class cost', _class_cost_test)
            lib.plot.plot('dev generated class cost', _gen_class_cost)
            lib.plot.plot('dev gen  cost', _dev_gen_cost_test)
            lib.plot.plot('dev gen accuracy', _dev_genAccuracy)
            lib.plot.plot('dev real accuracy', _dev_realAccuracy)

#            if iteration % 1000 == 999:
        if iteration % 100 == 99:
            generate_image(iteration)
            # Can add generate_good_images method in here if desired

        if (iteration < 10) or (iteration % 100 == 99):
            lib.plot.flush()

        lib.plot.tick()





Lossy conversion from float64 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.


{<tf.Tensor 'Placeholder_28:0' shape=(84, 3, 64, 64) dtype=int32>: array([[[[123, 120, 116, ..., 158, 164, 172],
         [121, 123, 126, ..., 140, 175, 194],
         [116, 127, 123, ..., 151, 183, 130],
         ...,
         [ 83,  73,  92, ..., 138, 111, 102],
         [114,  76,  90, ..., 123, 106,  99],
         [ 94,  76,  79, ..., 117, 110,  98]],

        [[ 87,  79,  73, ..., 107, 110, 114],
         [ 83,  81,  81, ...,  81, 117, 138],
         [ 75,  85,  78, ...,  86, 124,  78],
         ...,
         [ 64,  50,  69, ...,  90,  66,  66],
         [ 96,  56,  67, ...,  77,  68,  70],
         [ 76,  58,  59, ...,  72,  74,  70]],

        [[ 65,  59,  54, ...,  62,  63,  64],
         [ 60,  59,  60, ...,  41,  69,  87],
         [ 53,  61,  57, ...,  46,  80,  30],
         ...,
         [ 57,  42,  61, ...,  67,  47,  52],
         [ 86,  47,  59, ...,  54,  49,  56],
         [ 64,  48,  50, ...,  51,  58,  56]]],


       [[[167, 148, 152, ..., 174, 165, 171],
         

KeyboardInterrupt: 

In [ ]:
def image_grid(x, size=1):
    t = tf.unstack(x[:size * size], num=size*size, axis=0)
#     print(t)
    rows = [tf.concat(t[i*size:(i+1)*size], axis=0) 
            for i in range(size)]
#     print(rows)
    image = tf.concat(rows, axis=1)
    print(image[None])
    return image[None]

In [4]:
def reshape_row(arr):
    return reduce(lambda x, y: np.concatenate((x,y), axis=1), arr)

def reshape_col(arr):
    return reduce(lambda x, y: np.concatenate((x,y), axis=0), arr)

def arbitrary_rows_cols(arr, num_rows, num_cols, gray=False):
    num_images, height, width, depth, = arr.shape
    rows = []
    for i in range(num_rows):
        row_image = arr[i*num_cols:i*num_cols+num_cols]
        r_n, r_h, r_w, r_d = row_image.shape
        if row_image.shape[0] != num_cols:
            for _ in range(num_cols - row_image.shape[0]):
                row_image = np.concatenate((row_image, np.expand_dims(np.zeros((height, width, depth)), axis=0)), axis=0)
        row_image = reshape_row(row_image)
        rows.append(row_image)
    mosaic = reshape_col(rows)
    return mosaic

In [21]:
import numpy as np
import scipy.misc
# from scipy.misc import imsave

def save_images(X, save_path):
    # [0, 1] -> [0,255]
    print('started')
    if isinstance(X.flatten()[0], np.floating):
        print(X)
        X = (255.99*X).astype('uint8')
        print(X)
    print('dim: ',X.ndim)
    n_samples = X.shape[0]
    print('samples: ', n_samples)
    rows = int(np.sqrt(n_samples))
    print('rows: ', rows)
    while n_samples % rows != 0:
        print('1')
        rows -= 1

    nh, nw = rows, n_samples/rows
    nh, nw = int(nh), int(nw)

    if X.ndim == 2:
        X = np.reshape(X, (X.shape[0], int(np.sqrt(X.shape[1])), int(np.sqrt(X.shape[1]))))

    if X.ndim == 4:
        # BCHW -> BHWC
        X = X.transpose(0,2,3,1)
        print('X_new: ', X)
#         h, w = X[0].shape[:2]
        h = w = X[0].shape[1]
        print('h,w: ',  h,w)
        img = np.zeros((h*nh, w*nw, 3))
        print('img: ', img)
    elif X.ndim == 3:
        h, w = X[0].shape[:2]
        img = np.zeros((h*nh, w*nw))

    for n, x in enumerate(X):
        j = int(n/nw)
        i = int(n%nw)
        img[j*h:j*h+h, i*w:i*w+w] = x

#     imsave(save_path, img)
    imageio.imwrite(save_path,img)

In [11]:
im = imageio.imread('/home/ec2-user/SageMaker/portrait_landscape/landscapes/0.png', pilmode='RGB')

In [12]:
im = im.reshape(1,64,64,3)

In [13]:
# im.reshape(1, 3, 64, 64)

In [14]:
save_images(im, 'generated/samples_groundtruth.png')

Lossy conversion from float64 to uint8. Range [24.0, 250.0]. Convert image to uint8 prior to saving to suppress this warning.


started
dim:  4
samples:  1
rows:  1
X_new:  [[[[246 234 236]
   [242 232 233]
   [241 231 230]
   ...
   [240 228 228]
   [242 230 230]
   [241 230 228]]

  [[231 219 219]
   [227 217 216]
   [228 218 217]
   ...
   [226 214 214]
   [228 216 216]
   [228 216 216]]

  [[229 217 217]
   [219 209 208]
   [220 210 209]
   ...
   [226 214 214]
   [224 212 212]
   [225 213 213]]

  ...

  [[226 214 202]
   [226 214 202]
   [227 213 202]
   ...
   [233 216 206]
   [229 215 204]
   [225 211 200]]

  [[229 215 204]
   [229 215 204]
   [230 216 205]
   ...
   [226 208 196]
   [227 211 198]
   [228 211 201]]

  [[234 217 207]
   [232 215 205]
   [234 217 207]
   ...
   [233 215 203]
   [235 217 205]
   [232 216 203]]]]
h,w:  64 64
img:  [[[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 ...


In [ ]:
# im = ((im + 1.) * (255.99 / 2)).astype('int32')

In [29]:
im = image_grid(im.reshape(1,64,64,3))

NameError: name 'image_grid' is not defined

In [ ]:
im = tf.Session().run(im)

In [ ]:
from functools import reduce

In [ ]:
im = arbitrary_rows_cols(im, 1, 1, gray=False)
imageio.imwrite('generated/samples_groundtruth.png',im)

In [ ]:
im = tf.Session().run(im)

In [ ]:
im.shape

In [ ]:
imageio.imwrite('generated/samples_groundtruth.png',im)

In [ ]:
isinstance(0, np.floating)

In [ ]:
X = im.reshape(1, 3, 64, 64)
# X = im
# X = (255.99*X).astype('uint8')
n_samples = X.shape[0]
rows = int(np.sqrt(n_samples))
nh, nw = rows, n_samples/rows
nh = int(nh)
nw = int(nw)

In [ ]:
nh, nw 

In [ ]:
X = X.transpose(0,2,3,1)

In [ ]:
h, w = X[0].shape[:2]

In [ ]:
img = np.zeros((h*nh, w*nw, 3))

In [ ]:
for n, x in enumerate(X):
    j = int(n/nw)
    i = int(n%nw)
    print(j*h,j*h+h, i*w,i*w+w)
    img[j*h:j*h+h, i*w:i*w+w] = x
#     print(img[j*h:j*h+h, i*w:i*w+w])

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [ ]:
# tf.contrib.gan.eval.image_grid(
#     im.reshape((1, 3, 64, 64)),
#     grid_shape,
#     image_shape=(64, 64),
#     num_channels=3
# )

In [ ]:
img = img.astype(np.uint8)

In [ ]:
imageio.imwrite('generated/samples_groundtruth.png',img)

In [ ]:
save_images(im.reshape((1, 3, 64, 64)), 'generated/samples_groundtruth.png')

In [ ]:
im = ((im + 1.) * (255.99 / 2)).astype('int32')

In [ ]:
imageio.imwrite('generated/samples_groundtruth.png',im)

In [ ]:
import tensorflow as tf
tf.__version__

In [ ]:
# im = Image.open('/home/ec2-user/SageMaker/portrait_landscape/landscapes/0.png', mode='r')

In [ ]:
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
img = mpimg.imread('/home/ec2-user/SageMaker/portrait_landscape/landscapes/0.png')
imgplot = plt.imshow(im)
plt.savefig('foo.png')
plt.show()


In [ ]:
im.save.im.reshape((1, 3, 64, 64), 'generated/samples_groundtruth.png')

In [ ]:
train_gen, dev_gen = lib.wikiartGenre.load(BATCH_SIZE, CLASSES)

In [ ]:
import os, sys
sys.path.append(os.getcwd())

import tflib.save_images
import tflib.wikiartGenre
import tflib.plot
import imageio

from GANgogh import *

from tflib.wikiartGenre import get_style

DEVICES = ['/gpu:{}'.format(i) for i in range(N_GPUS)]

ITERS = 2000 #200000  # How many iterations to train for
PREITERATIONS = 20# 2000  # Number of preiteration training cycles to run


Generator, Discriminator = GeneratorAndDiscriminator()

with tf.Session(config=tf.ConfigProto(allow_soft_placement=True)) as session:
    all_real_data_conv = tf.placeholder(tf.int32, shape=[BATCH_SIZE, 3, 64, 64])
    all_real_label_conv = tf.placeholder(tf.int32, shape=[BATCH_SIZE, CLASSES])

    generated_labels_conv = tf.placeholder(tf.int32, shape=[BATCH_SIZE, CLASSES])
    sample_labels_conv = tf.placeholder(tf.int32, shape=[BATCH_SIZE, CLASSES])

    split_real_data_conv = tf.split(all_real_data_conv, len(DEVICES))
    split_real_label_conv = tf.split(all_real_label_conv, len(DEVICES))
    split_generated_labels_conv = tf.split(generated_labels_conv, len(DEVICES))
    split_sample_labels_conv = tf.split(sample_labels_conv, len(DEVICES))

    gen_costs, disc_costs = [], []

    for device_index, (device, real_data_conv, real_label_conv) in enumerate(
            zip(DEVICES, split_real_data_conv, split_real_label_conv)):

        with tf.device(device):
            real_data = tf.reshape(2 * ((tf.cast(real_data_conv, tf.float32) / 255.) - .5),
                                   [BATCH_SIZE // len(DEVICES), OUTPUT_DIM])
            print(real_data)
            _x, _y = next(train_gen())
            _x_r = session.run(real_data, feed_dict={all_real_data_conv: _x})
            _x_r = ((_x_r + 1.) * (255.99 / 2)).astype('int32')
            lib.save_images.save_images(_x_r.reshape((BATCH_SIZE, 3, 64, 64)), 'generated/samples_groundtruth.png')

In [ ]:
tf.test.gpu_device_name()

In [ ]:
_x, _y = next(train_gen())
_x_r = session.run(real_data, feed_dict={all_real_data_conv: _x})
_x_r = ((_x_r + 1.) * (255.99 / 2)).astype('int32')
lib.save_images.save_images(_x_r.reshape((BATCH_SIZE, 3, 64, 64)), 'generated/samples_groundtruth.png')

In [ ]:
# pip install -r requirements.txt

In [ ]:
pip freeze | grep tensorflow

In [ ]:
pip uninstall tensorflow

In [ ]:
pip install tensorflow==1.15

In [ ]:
import tensorflow as tf

In [ ]:
tf.test.gpu_device_name()

In [ ]:
tf.__version__